In [57]:
#importy
import sys
import arcpy

In [58]:
# włączenie nadpisywania plików
arcpy.env.overwriteOutput=True

In [59]:
# sprawdzanie rozszerzeń
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")

'CheckedOut'

In [60]:
# ustawienia środowiska pracy
arcpy.env.workspace  = r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result'

# ustawienie obszaru pracy (prostokąta otaczającego)
arcpy.env.extent = r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\MyProject\buffer.shp'

# ustawienie układu współrzędnych
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")

# ustawienie wielkości komórki
arcpy.env.cellSize = 1

wczytanie warstw

In [61]:
buf = "buffer"
jeziora = "jeziora"
gazociag = "gazociag"
rzeki = "rzeki"
drogi = "drogi"
tory = "tory"
lasy_temp = "lasy"
budynki_temp = "budynki"
granica = "granica_swieradow"
raster = "gmina_swieradow_raster"
gleby='gleby_gm_swieradow'
dzialki_equal = 'dzialki_swieradow_equal'
dzialki_diff = 'dzialki_swieradow_diff'

selekcja potrzebnych warstw do kryteriów

In [62]:
# wylaczenie dodawania warstw
arcpy.env.addOutputsToMap = False
#selekcja budynkow mieszkalnych przy pomocy atrubutu X_KOD
budynki = arcpy.management.SelectLayerByAttribute(budynki_temp, "NEW_SELECTION", "X_KOD IN ('BUBD02','BUBD03', 'BUBD04')", None)
arcpy.management.CopyFeatures(budynki, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\budynki_mieszkalne_selected", '', None, None, None)
budynki_mieszkalne = 'budynki_mieszkalne_selected'
arcpy.SelectLayerByAttribute_management(budynki, "CLEAR_SELECTION")

# selekcja lasow i zagajnikow przy pomocy atrubutu RODZAJ
lasy = arcpy.management.SelectLayerByAttribute(lasy_temp, "NEW_SELECTION", "RODZAJ IN ('las', 'zagajnik')", None)
arcpy.management.CopyFeatures(lasy, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\lasy_selected", '', None, None, None)
lasy = 'lasy_selected'
arcpy.SelectLayerByAttribute_management(lasy, "CLEAR_SELECTION")

# selekcja hoteli przy pomocy atrubutu X_KOD
hotele = arcpy.management.SelectLayerByAttribute(budynki_temp, "NEW_SELECTION", "X_KOD IN ('BUBD05', 'BUBD06')", None)
arcpy.management.CopyFeatures(hotele, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\hotele_selected", '', None, None, None)
hotele = 'hotele_selected'
arcpy.SelectLayerByAttribute_management(hotele, "CLEAR_SELECTION")

<Result 'hotele_selected_Layer5'>

Kryterium nr. 1 / odległość od kokurencji min. 400 m / im dalej tym korzystniej 

In [63]:
# obliczenie odleglosci euklidesowej
distance = arcpy.sa.EucDistance(hotele)

In [64]:
# wycięcie terenu gminy z prostokata otaczajacego
extracted_by_mask = arcpy.sa.ExtractByMask(distance, granica, "INSIDE")

In [65]:
# transformacja skali rastra na [0:1]
arcpy.env.addOutputsToMap = False
result1 = arcpy.sa.FuzzyMembership(extracted_by_mask, arcpy.sa.FuzzyLinear(380, 420))
result1.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_1.tif')

Kryterium nr. 2 / odległość od budynków mieszkalnych od 25m do 150m ma watość 1 a wraz z oddalaniem spada wartość 

In [66]:
arcpy.env.addOutputsToMap = False
# obliczenie odleglosci euklidesowej
distance2 = arcpy.sa.EucDistance(budynki_mieszkalne)

In [67]:
# wycięcie terenu gminy z prostokata otaczajacego
extracted_by_mask2 = arcpy.sa.ExtractByMask(distance2, granica, "INSIDE")

In [68]:
# transformacja skali rastra na [0:1]
result2_vol1 = arcpy.sa.FuzzyMembership(extracted_by_mask2, arcpy.sa.FuzzyLinear(5, 25))
result2_vol2 = arcpy.sa.FuzzyMembership(extracted_by_mask2, arcpy.sa.FuzzyLinear(180, 150))
arcpy.env.addOutputsToMap = False
result2 = result2_vol1 * result2_vol2
result2.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_2.tif')

Kryterium nr. 3 / odległość od istniejących dróg / od 15 do 100m ma wartość 1 a wraz z oddalaniem wartość spada do 0

In [69]:
arcpy.env.addOutputsToMap = False
# obliczenie odleglosci euklidesowej
distance3 = arcpy.sa.EucDistance(drogi)

In [70]:
# wyciecie terenu gminy z prostokata otaczajacego
extracted_by_mask3 = arcpy.sa.ExtractByMask(distance3, granica, "INSIDE")

In [71]:
# transformacja skali rastra na [0:1]
result3_vol1 = arcpy.sa.FuzzyMembership(extracted_by_mask3, arcpy.sa.FuzzyLinear(5, 15))
result3_vol2 = arcpy.sa.FuzzyMembership(extracted_by_mask3, arcpy.sa.FuzzyLinear(130, 100))
arcpy.env.addOutputsToMap = False
result3 = result3_vol1 * result3_vol2
result3.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_3.tif')

Kryterium nr. 4 / odległość od rzek i zbiorników wodnych / nieprzekraczalna strefa ochronna poniżej 20m ma wartość 1 a wraz z oddalaniem wartość spada do 0

In [72]:
arcpy.env.addOutputsToMap = False
# obliczenie odleglosci euklidesowej
distance4_jez = arcpy.sa.EucDistance(jeziora)
distance4_rze = arcpy.sa.EucDistance(rzeki)

In [73]:
# wyciecie terenu gminy z prostokata otaczajacego
extracted_by_mask4_jez = arcpy.sa.ExtractByMask(distance4_jez, granica, "INSIDE")
extracted_by_mask4_rze = arcpy.sa.ExtractByMask(distance4_rze, granica, "INSIDE")
max_value_jez = extracted_by_mask4_jez.maximum * 1.1
max_value_rze = extracted_by_mask4_rze.maximum * 1.1

In [74]:
arcpy.env.addOutputsToMap =False
# kryterium ostre
rzeki_ostre = extracted_by_mask4_rze > 20
jeziora_ostre = extracted_by_mask4_jez > 20
rzeki_ostre.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_4_rzeki_ostre.tif')
jeziora_ostre.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_4_jeziora_ostre.tif')


In [75]:
arcpy.env.addOutputsToMap = False
# kryterium rozmyte
#transformacja skali rastra na [0:1]
result4_1_jez = arcpy.sa.FuzzyMembership(extracted_by_mask4_jez, arcpy.sa.FuzzyLinear(20, 20.1))
result4_1_rze = arcpy.sa.FuzzyMembership(extracted_by_mask4_rze, arcpy.sa.FuzzyLinear(20, 20.1))
result4_jez = arcpy.sa.FuzzyMembership(extracted_by_mask4_jez, arcpy.sa.FuzzyLinear(max_value_jez, 20))
result4_rze = arcpy.sa.FuzzyMembership(extracted_by_mask4_rze, arcpy.sa.FuzzyLinear(max_value_rze, 20))
arcpy.env.addOutputsToMap = False
result4_rozmyty_jez = result4_jez * result4_1_jez
result4_rozmyty_rze = result4_1_rze * result4_rze
result4_rozmyty_jez.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_4r_jeziora.tif')
result4_rozmyty_rze.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_4r_rzeki.tif')

Kryterium nr. 5 / pokrycie terenu / nie w lesie / selekcja z bdotu lasów i zagajników / jeśli teren znajduje się w lesie ma wartość 0 jeśli nie to ma wartość 1


In [76]:
arcpy.env.addOutputsToMap = False
# obliczenie odleglosci euklidesowej
distance5 = arcpy.sa.EucDistance(lasy)

In [77]:
# wyciecie terenu gminy z prostokata otaczajacego
extracted_by_mask5 = arcpy.sa.ExtractByMask(distance5, granica, "INSIDE")

In [78]:
# transformacja skali rastra na [0:1]
arcpy.env.addOutputsToMap = False
result5 = arcpy.sa.FuzzyMembership(extracted_by_mask5, arcpy.sa.FuzzyLinear(0, 0.1))
result5.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_5.tif')


Kryterium nr. 6 / nachylenie stoków - maksymalnie 20% / dla 20 % wartość 0 im mniej tym zwiększa się wartość do 1

In [79]:
arcpy.env.addOutputsToMap = False
# obliczenie Slope
nachylenie = arcpy.sa.Slope(raster, "PERCENT_RISE")

In [80]:
# wyciecie terenu gminy z prostokata otaczajacego
extracted_by_mask6 = arcpy.sa.ExtractByMask(nachylenie, granica, "INSIDE")

In [81]:
# transformacja skali rastra na [0:1]
arcpy.env.addOutputsToMap = False
result6 = arcpy.sa.FuzzyMembership(extracted_by_mask6, arcpy.sa.FuzzyLinear(21, 18))
result6.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_6.tif')

Kryterium nr. 7 / dostęp światła słonecznego / stoki południowe (SW-SE) mają wartość 1 a potem kryt rozmyte 

In [82]:
# obliczenie Aspect
arcpy.env.addOutputsToMap = False
naslonecznienie = arcpy.sa.Aspect(raster)

In [83]:
# wyciecie terenu gminy z prostokata otaczajacego
extracted_by_mask7 = arcpy.sa.ExtractByMask(naslonecznienie, granica, "INSIDE")

In [84]:
result7_vol1 = arcpy.sa.FuzzyMembership(extracted_by_mask7, arcpy.sa.FuzzyLinear(90, 112.5))
result7_vol2 = arcpy.sa.FuzzyMembership(extracted_by_mask7, arcpy.sa.FuzzyLinear(270, 247.5))
temp = result7_vol1 * result7_vol2
# funkcja Con aby dodac też płaskie tereny (input raster, jesli value = -1 to przyjmij wartos 1, jesli value != -1 to przyjmij wartosc z mnozenia, value)
arcpy.env.addOutputsToMap = False
result7 = arcpy.ia.Con(extracted_by_mask7, 1, temp, "VALUE = -1")
result7.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_7.tif')

Kryterium nr. 8 / odleglosc od gazociagu / min 25 m kryterium rozmyte im dalej tym lepiej

In [85]:
# obliczenie odleglosci euklidesowej
arcpy.env.addOutputsToMap = False
distance8 = arcpy.sa.EucDistance(gazociag)

In [86]:
# wyciecie terenu gminy z prostokata otaczajacego
extracted_by_mask8 = arcpy.sa.ExtractByMask(distance8, granica, "INSIDE")

In [87]:
# transformacja skali rastra na [0:1]
arcpy.env.addOutputsToMap =False
result8_ostre = arcpy.sa.FuzzyMembership(extracted_by_mask8, arcpy.sa.FuzzyLinear(25, 25.1))
result8_rozmyte = arcpy.sa.FuzzyMembership(extracted_by_mask8, arcpy.sa.FuzzyLinear(25, 55))
result8_ostre.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_8_ostre.tif')
result8_rozmyte.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kryt_8_rozmyte.tif')

Kryterium nr. 9  OSTRE/ użytkowanie terenu / gleby nieprzydatne [G,E,M,T,F,Fb,Fc,FG] reszta przydatna

In [88]:
# ustawia wartości kolumny VALUE na 1 jeśli gleba jest poza tablicą gleb nieprzydatnych
arcpy.management.CalculateField("gleby_gm_swieradow", "VALUE", "gleba(!TYP!)", "PYTHON3", """def gleba(x):
    return x not in ['G', 'E', 'M', 'T', 'F', 'Fb', 'Fc', 'FG']
 """, "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'gleby_gm_swieradow'>

In [89]:
arcpy.env.addOutputsToMap =False
result9 = arcpy.conversion.PolygonToRaster("gleby_gm_swieradow", 'VALUE', 'kryt_9.tif')

In [90]:
arcpy.env.addOutputsToMap =False
# łączenie kryteriów rozmytych, ta sama waga
waga = 1/8
result_rozmyte_equal = arcpy.ia.WeightedSum(arcpy.sa.WSTable([[result1, "VALUE", waga], [result2, "VALUE", waga], [result3, "VALUE", waga], [result4_rozmyty_jez, "VALUE", waga], [result4_rozmyty_rze, "VALUE", waga], [result6, "VALUE", waga], [result7, "VALUE", waga], [result8_rozmyte, "VALUE", waga]]))
# łączenie kryteriów rozmytych, różne wagi
result_rozmyte_diff = arcpy.ia.WeightedSum(arcpy.sa.WSTable([[result1, "VALUE", 0.20], [result2, "VALUE", 0.10], [result3, "VALUE", 0.15], [result4_rozmyty_jez, "VALUE", 0.10], [result4_rozmyty_rze, "VALUE", 0.10], [result6, "VALUE", 0.05], [result7, "VALUE", 0.10], [result8_rozmyte, "VALUE", 0.20]]))

In [91]:
arcpy.env.addOutputsToMap =False
# łączenie kryteriów ostrych
result_ostry = arcpy.sa.FuzzyOverlay([rzeki_ostre, jeziora_ostre, result5, result8_ostre, result9])

In [92]:
# łączenie kryteriów aby otrzymac raster wynikowy
result_equal = result_ostry * result_rozmyte_equal
result_equal.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\wynik_rowne.tif')

result_diff = result_ostry * result_rozmyte_diff
result_diff.save(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\wynik_rozne.tif')

In [94]:
# merge warstw pokrycia terenu, aby utworzyć mapę kosztów
tab = ['PL.PZGiK.337.0210__OT_PTZB_A', 'PL.PZGiK.337.0210__OT_PTWZ_A', 'PL.PZGiK.337.0210__OT_PTWP_A', 'PL.PZGiK.337.0210__OT_PTUT_A', 'PL.PZGiK.337.0210__OT_PTTR_A', 'PL.PZGiK.337.0210__OT_PTSO_A', 'PL.PZGiK.337.0210__OT_PTRK_A', 'PL.PZGiK.337.0210__OT_PTPL_A', 'PL.PZGiK.337.0210__OT_PTNZ_A', 'PL.PZGiK.337.0210__OT_PTLZ_A', 'PL.PZGiK.337.0210__OT_PTKM_A', 'PL.PZGiK.337.0210__OT_PTGN_A']
pokrycie_terenu = arcpy.management.Merge(tab, 'pokrycie_terenu')

In [95]:
# tworzenie mapy kosztow
arcpy.management.CalculateField("pokrycie_terenu", "koszt", "temp(!X_KOD!)", "PYTHON3", """def temp(kod):
    dict = {'PTWP01':0,
    'PTWP02':200,
    'PTWP03':0,
    'PTZB01':200,
    'PTZB02':100,
    'PTZB03':200,
    'PTZB04':200,
    'PTZB05':50,
    'PTLZ01':100,
    'PTLZ02':50,
    'PTLZ03':50,
    'PTRK':15,
    'PTRK02':15,
    'PTUT01':0,
    'PTUT02':90,
    'PTUT03':100,
    'PTUT04':20,
    'PTUT05':20,
    'PTTR01':20,
    'PTTR02':1,
    'PTKM01':100,
    'PTKM02':200,
    'PTKM03':0,
    'PTKM04':0,
    'PTGN01':1,
    'PTGN02':1,
    'PTGN03':1,
    'PTGN04':1,
    'PTPL01':50,
    'PTSO01':0,
    'PTSO02':0,
    'PTWZ01':0,
    'PTWZ02':0,
    'PTNZ01':150,
    'PTNZ02':150}

    
    return dict[kod]

""", "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'C:\\Users\\weron\\Desktop\\studia\\5sem\\analiza_przestrzenna\\swieradow\\new\\result\\pokrycie_terenu'>

In [96]:
arcpy.management.Delete(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\mapa_kosztow')
mapa_kosztow = arcpy.conversion.PolygonToRaster(pokrycie_terenu, "koszt", 'mapa_kosztow')

In [97]:
# reklasyfikacja aby użyć table intersection
reclass_diff = arcpy.sa.Reclassify(result_diff, "VALUE", arcpy.sa.RemapRange([[0,0.75,0],[0.75,result_diff.maximum,1]]))
reclass_equal = arcpy.sa.Reclassify(result_equal, "VALUE", arcpy.sa.RemapRange([[0,0.75,0],[0.75,result_equal.maximum,1]]))

In [98]:
arcpy.env.addOutputsToMap = False
# zamiana rastra na postać wektorową bo table intersection przyjumuje tylko dane wektorowe
polygon_diff = arcpy.conversion.RasterToPolygon(reclass_diff, 'polygon_diff', 'NO_SIMPLIFY')
polygon_equal = arcpy.conversion.RasterToPolygon(reclass_equal, 'polygon_equal', 'NO_SIMPLIFY')

In [99]:
arcpy.env.addOutputsToMap = True
# selekcja warstw wektorowych z wartoscia 1
select_diff = arcpy.management.SelectLayerByAttribute(polygon_diff, "NEW_SELECTION", "gridcode = 1", None)
arcpy.management.CopyFeatures(select_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\select_diff", '', None, None, None)
select_diff = 'select_diff'
arcpy.SelectLayerByAttribute_management(select_diff, "CLEAR_SELECTION")

select_equal = arcpy.management.SelectLayerByAttribute(polygon_equal, "NEW_SELECTION", "gridcode = 1", None)
arcpy.management.CopyFeatures(select_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\select_equal", '', None, None, None)
select_equal = 'select_equal'
arcpy.SelectLayerByAttribute_management(select_equal, "CLEAR_SELECTION")

<Result 'select_equal'>

In [100]:
# tabulate intersection
arcpy.management.Delete(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\MyProject1.gdb\table_diff')
arcpy.analysis.TabulateIntersection(dzialki_diff, "ID_DZIALKI", select_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\MyProject1.gdb\table_diff")

arcpy.management.Delete(r'C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\MyProject1.gdb\table_equal')
arcpy.analysis.TabulateIntersection(dzialki_equal, "ID_DZIALKI", select_equal, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\MyProject1.gdb\table_equal")

<Result 'C:\\Users\\weron\\Desktop\\studia\\5sem\\analiza_przestrzenna\\swieradow\\new\\MyProject1.gdb\\table_equal'>

In [101]:
arcpy.management.AddJoin(dzialki_diff, "ID_DZIALKI", "table_diff", "ID_DZIALKI")


<Result 'dzialki_swieradow_diff'>

In [102]:
arcpy.management.AddJoin(dzialki_equal, "ID_DZIALKI", "table_equal", "ID_DZIALKI")

<Result 'dzialki_swieradow_equal'>

In [103]:
# wybor dzialek ktorych pokrycie wynosi wiecej niz 70%
arcpy.env.addOutputsToMap = False
dzialki_70_diff = arcpy.management.SelectLayerByAttribute(dzialki_diff, "NEW_SELECTION", "PERCENTAGE >= 70", None)
arcpy.management.CopyFeatures(dzialki_70_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\dzialki_70_diff", '', None, None, None)
dzialki_70_diff = 'dzialki_70_diff'
arcpy.SelectLayerByAttribute_management(dzialki_70_diff, "CLEAR_SELECTION")

dzialki_70_equal = arcpy.management.SelectLayerByAttribute(dzialki_equal, "NEW_SELECTION", "PERCENTAGE >= 70", None)
arcpy.management.CopyFeatures(dzialki_70_equal, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\dzialki_70_equal", '', None, None, None)
dzialki_70_equal = 'dzialki_70_equal'
arcpy.SelectLayerByAttribute_management(dzialki_70_equal, "CLEAR_SELECTION")

<Result 'dzialki_70_equal_Layer5'>

In [104]:
# rozcieranie granic pomiędzy dzialkami
dissolve_diff = arcpy.management.Dissolve(dzialki_70_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\MyProject1.gdb\dissolve_diff", None, None,"SINGLE_PART")
dissolve_equal = arcpy.management.Dissolve(dzialki_70_equal, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\MyProject1.gdb\dissolve_equal", None, None,"SINGLE_PART")

In [105]:
arcpy.env.addOutputsToMap = True
selected_area_diff = arcpy.management.SelectLayerByAttribute(dissolve_diff, "NEW_SELECTION", "Shape_Area >= 10000", None)
arcpy.management.CopyFeatures(selected_area_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\selected_area_diff", '', None, None, None)
selected_area_diff = 'selected_area_diff'
arcpy.SelectLayerByAttribute_management(selected_area_diff, "CLEAR_SELECTION")

selected_area_equal = arcpy.management.SelectLayerByAttribute(dissolve_equal, "NEW_SELECTION", "Shape_Area >= 10000", None)
arcpy.management.CopyFeatures(selected_area_equal, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\selected_area_equal", '', None, None, None)
selected_area_equal = 'selected_area_equal'
arcpy.SelectLayerByAttribute_management(selected_area_equal, "CLEAR_SELECTION")

<Result 'selected_area_equal'>

In [106]:
# obliczenie współczynnika kształtu dla działek z równą wagą
arcpy.management.CalculateField(selected_area_equal, "ksztalt", "licz(!Shape_Area!,!Shape_Leng!)", "PYTHON3", """def licz(pole, obwod):
     r = obwod / (2 * math.pi)
     pole_kola = math.pi * r ** 2
     wskaznik = math.sqrt(pole/pole_kola)
     return wskaznik""", "DOUBLE", "NO_ENFORCE_DOMAINS")


# obliczenie współczynnika kształtu dla działek z różną wagą
arcpy.management.CalculateField(selected_area_diff, "ksztalt", "licz(!Shape_Area!,!Shape_Leng!)", "PYTHON3", """def licz(pole, obwod):
     r = obwod / (2 * math.pi)
     pole_kola = math.pi * r ** 2
     wskaznik = math.sqrt(pole/pole_kola)
     return wskaznik""", "DOUBLE", "NO_ENFORCE_DOMAINS")

<Result 'selected_area_diff'>

In [107]:
# reklasyfikacja mapy kosztow, która nie ma wartości 0, tylko no data
mapa_kosztow_ok = arcpy.sa.Reclassify(mapa_kosztow, "VALUE", arcpy.sa.RemapValue([[0,'NODATA'],[1,1],[15,15],[20,20],[50,50],[100,100],[150,150]]))

In [108]:
# mapa kosztow skumulowanych i mapa kierunkow
arcpy.env.addOutputsToMap = False
cost_distance_diff = arcpy.sa.CostDistance(selected_area_diff, mapa_kosztow_ok, None, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kierunki_diff.tif", None, None, None, None, ''); 
cost_distance_diff.save(r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\mapa_kosztow_skumulowanych_diff.tif")

In [109]:
cost_distance_equal = arcpy.sa.CostDistance(selected_area_equal, mapa_kosztow_ok, None, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kierun_equal.tif", None, None, None, None, ''); 
cost_distance_equal.save(r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\mapa_kosztow_skumulowanych_equal.tif")

In [110]:
# przyłącze dla równych wag
arcpy.env.addOutputsToMap = True
przylacze_equal = arcpy.sa.CostPath(gazociag, cost_distance_equal, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kierun_equal.tif", "BEST_SINGLE", "Id", "INPUT_RANGE"); 
przylacze_equal.save(r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\przylacze_equal.tif")

In [111]:
# przyłącze dla różnych wag
przylacze_diff = arcpy.sa.CostPath(gazociag, cost_distance_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\kierunki_diff.tif", "BEST_SINGLE", "Id", "INPUT_RANGE"); 
przylacze_diff.save(r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\przylacze_diff.tif")

In [ ]:
# ostateczny wybór działki z najbardziej zwartym kształtem (największy współczynnik kształtu)
final_wynik_diff = arcpy.management.SelectLayerByAttribute(selected_area_diff, "NEW_SELECTION", "ksztalt = (SELECT MAX(ksztalt) FROM selected_area_diff)", None)
arcpy.management.CopyFeatures(final_wynik_diff, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\final_wynik_diff", '', None, None, None)
final_wynik_equal = arcpy.management.SelectLayerByAttribute(selected_area_equal, "NEW_SELECTION", "ksztalt = (SELECT MAX(ksztalt) FROM selected_area_equal)", None)
arcpy.management.CopyFeatures(final_wynik_equal, r"C:\Users\weron\Desktop\studia\5sem\analiza_przestrzenna\swieradow\new\result\final_wynik_equal", '', None, None, None)